In [2]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [24]:
engine = create_engine('sqlite:///chinook.db')
engineDW = create_engine('sqlite:///DW_SalesMusic.db')

# Extract

In [27]:
metadata = MetaData()

metadata.create_all(engine)
metadata.create_all(engineDW)

inspector = inspect(engine)
inspectorDW = inspect(engineDW)

# inspección de emplooyes

In [19]:
inspector.get_columns('employees')

[{'name': 'EmployeeId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'LastName',
  'type': NVARCHAR(length=20),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'FirstName',
  'type': NVARCHAR(length=20),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Title',
  'type': NVARCHAR(length=30),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'ReportsTo',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'BirthDate',
  'type': DATETIME(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'HireDate',
  'type': DATETIME(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Address',
  'type': NVARCHAR(length=70),
 

# inspección de albums

In [20]:
inspector.get_columns('albums')

[{'name': 'AlbumId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Title',
  'type': NVARCHAR(length=160),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'ArtistId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

# inspección de media_types

In [21]:
inspector.get_columns('media_types')

[{'name': 'MediaTypeId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Name',
  'type': NVARCHAR(length=120),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

# Inspección de genres

In [22]:
inspector.get_columns('genres')

[{'name': 'GenreId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Name',
  'type': NVARCHAR(length=120),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

# Inspección de playlists

In [23]:
inspector.get_columns('playlists')

[{'name': 'PlaylistId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Name',
  'type': NVARCHAR(length=120),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

# Transform

## dim_employees

In [6]:
import pandas as pd
df_employees=pd.read_sql_query("""SELECT LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, 
City, State, Country, PostalCode, Phone, Fax, Email
FROM employees;
 """, con=engine.connect())
df_employees.head()


,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


# dim_location

In [8]:
df_location=pd.read_sql_query("""SELECT BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode
FROM invoices;
 """, con=engine.connect())
df_location=df_location.drop_duplicates(['BillingAddress'], keep='first')
df_location.head()

,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode
0,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174
1,Ullevålsveien 14,Oslo,None,Norway,0171
2,Grétrystraat 63,Brussels,None,Belgium,1000
3,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7
4,69 Salem Street,Boston,MA,USA,2113


# dim_time

In [9]:
df_time=pd.read_sql_query("""SELECT InvoiceDate
FROM invoices;
 """, con=engine.connect())
df_time=df_time.drop_duplicates(['InvoiceDate'], keep='first')
df_time.head()

,InvoiceDate
0,2009-01-01 00:00:00
1,2009-01-02 00:00:00
2,2009-01-03 00:00:00
3,2009-01-06 00:00:00
4,2009-01-11 00:00:00


# dim_tracks

In [10]:

df_tracks=pd.read_sql_query("""SELECT TrackId,Name,MediaTypeID,GenreId,Composer,Milliseconds,Bytes,UnitPrice
FROM tracks;
 """, con=engine.connect())
df_tracks.head()

,TrackId,Name,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


# dim_customers

In [11]:
df_customers=pd.read_sql_query("""SELECT CustomerId,Firstname,LastName,Company,Address,City,State,Country,PostalCode,
Phone,Fax,Email,SupportRepId
FROM customers;
 """, con=engine.connect())
df_customers.head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


# dim_invoice

In [12]:
df_invoices=pd.read_sql_query("""SELECT InvoiceID, Total
FROM invoices;
 """, con=engine.connect())
df_invoices.head()

,InvoiceId,Total
0,1,1.98
1,2,3.96
2,3,5.94
3,4,8.91
4,5,13.86


# dim_artists

In [13]:
df_artists=pd.read_sql_query("""SELECT ArtistID, Name
FROM artists;
 """, con=engine.connect())
df_artists.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


# dim_playlists

In [14]:
df_playlists=pd.read_sql_query("""SELECT PlaylistID, Name
FROM playlists;
 """, con=engine.connect())
df_playlists.head()

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


# dim_genre

In [15]:
df_genres=pd.read_sql_query("""SELECT GenreID, Name
FROM genres;
 """, con=engine.connect())
df_genres.head()

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll


# dim_media_types

In [16]:
df_mediatypes=pd.read_sql_query("""SELECT MediaTypeID, Name
FROM media_types;
 """, con=engine.connect())
df_mediatypes.head()

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


# dim_albums

In [17]:
df_albums=pd.read_sql_query("""SELECT AlbumId, Title
FROM albums;
 """, con=engine.connect())
df_albums.head()

,AlbumId,Title
0,1,For Those About To Rock We Salute You
1,2,Balls to the Wall
2,3,Restless and Wild
3,4,Let There Be Rock
4,5,Big Ones


# Load

## Funciones de carga 

In [6]:
def updateData(name_table,data_db,name_columns):
	print(name_columns)
	aux_values='?'
	aux_data=[]
	aux_columns=name_columns[0]
	aux_data.append(list(data_db[name_columns[0]]))
	entities=[]
	aux_entities=[]
	for i in range(len(name_columns)-1):
		aux_values+=',?'
		aux_columns+=','+name_columns[i+1]
		aux_data.append(list(data_db[name_columns[i+1]]))
	for i in range(len(aux_data[0])):
		for j in range(len(aux_data)):
			aux_entities.append(aux_data[j][i])
		entities.append(aux_entities)
		aux_entities=[]
	with engineDW.connect() as con:
		for i in entities:
			con.execute('INSERT INTO '+name_table+'('+aux_columns+') VALUES('+aux_values+')', i)
	con.close()


## dim_employees 

In [33]:
updateData('dim_employees',df_employees,list(df_employees.columns.values))

['LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']


## dim_albums

In [68]:
updateData('dim_albums',df_albums,list(df_albums.columns.values))

['AlbumId', 'Title']


## dim_media_types

In [70]:
updateData('dim_media_types',df_mediatypes,list(df_mediatypes.columns.values))

['MediaTypeId', 'Name']


## dim_genres

In [71]:
updateData('dim_genres',df_genres,list(df_genres.columns.values))

['GenreId', 'Name']


## dim_playlists

In [72]:
updateData('dim_playlists',df_playlists,list(df_playlists.columns.values))

['PlaylistId', 'Name']


## dim_artists

In [73]:
updateData('dim_artists',df_artists,list(df_artists.columns.values))

['ArtistId', 'Name']


## dim_invoice

In [74]:
updateData('dim_invoice',df_invoices,list(df_invoices.columns.values))

['InvoiceId', 'Total']


## dim_customers

In [76]:
updateData('dim_customers',df_customers,list(df_customers.columns.values))

['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']


## dim_tracks

In [ ]:
updateData('dim_tracks',df_tracks,list(df_tracks.columns.values))

['TrackId', 'Name', 'MediaTypeId', 'GenreId', 'Composer', 'Milliseconds', 'Bytes', 'UnitPrice']


## dim_time

In [8]:
updateData('dim_time',df_time,list(df_time.columns.values))

['InvoiceDate']


## dim_location

In [12]:
updateData('dim_location',df_location,list(df_location.columns.values))

['BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode']


# Invoice_Fact

In [7]:
df_AuxInvoiceId=pd.read_sql_query("""SELECT InvoiceId FROM invoice_items;
 """, con=engine.connect())
df_AuxInvoiceId.head()

df_CustInv=pd.read_sql_query("""SELECT InvoiceId, CustomerId FROM invoices;
 """, con=engine.connect())

aux=list(df_AuxInvoiceId['InvoiceId'])
auxCustomer=[]
for i in aux:
    aux2=df_CustInv.index[df_CustInv['InvoiceId'] == i].tolist()
    auxCustomer.append(df_CustInv['CustomerId'][aux2[0]])

df_timeInv=pd.read_sql_query("""SELECT InvoiceId, InvoiceDate FROM invoices;
 """, con=engine.connect())
aux=list(df_AuxInvoiceId['InvoiceId'])
auxTime1=[]
for i in aux:
    aux22=df_CustInv.index[df_CustInv['InvoiceId'] == i].tolist()
    auxTime1.append(df_timeInv['InvoiceDate'][aux22[0]])
df_dim_time=pd.read_sql_query("""SELECT TimeID, InvoiceDate FROM dim_time;
 """, con=engineDW.connect())
auxTime=[]
for i in auxTime1:
    aux22=df_dim_time.index[df_dim_time['InvoiceDate'] == i].tolist()
    auxTime.append(df_dim_time['TimeID'][aux22[0]])

df_locationInv=pd.read_sql_query("""SELECT InvoiceId, BillingAddress FROM invoices;
 """, con=engine.connect())
aux=list(df_AuxInvoiceId['InvoiceId'])
auxTime1=[]
for i in aux:
    aux22=df_locationInv.index[df_locationInv['InvoiceId'] == i].tolist()
    auxTime1.append(df_locationInv['BillingAddress'][aux22[0]])
df_dim_location=pd.read_sql_query("""SELECT LocationID, BillingAddress FROM dim_location;
 """, con=engineDW.connect())
auxLocation=[]
for i in auxTime1:
    aux22=df_dim_location.index[df_dim_location['BillingAddress'] == i].tolist()
    auxLocation.append(df_dim_location['LocationID'][aux22[0]])
    
df_AuxTrackId=pd.read_sql_query("""SELECT TrackId FROM invoice_items;
 """, con=engine.connect())
df_AuxTrackId.head()

df_tracksP=pd.read_sql_query("""SELECT TrackId, AlbumId FROM tracks;
 """, con=engine.connect())
df_album=pd.read_sql_query("""SELECT AlbumId, Title, ArtistId FROM albums;
 """, con=engine.connect())
aux=list(df_AuxTrackId['TrackId'])
auxAlbum=[]
for i in aux:
    aux22=df_tracksP.index[df_tracksP['TrackId'] == i].tolist()
    auxAlbum.append(df_tracksP['AlbumId'][aux22[0]])


df_tracksPlay=pd.read_sql_query("""SELECT PlaylistId, TrackId FROM playlist_track;
 """, con=engine.connect())
aux=list(df_AuxTrackId['TrackId'])
auxPlaylist=[]
for i in aux:
    aux22=df_tracksPlay.index[df_tracksPlay['TrackId'] == i].tolist()
    auxPlaylist.append(df_tracksPlay['PlaylistId'][aux22[0]])    

    
    df_tracksPlay=pd.read_sql_query("""SELECT PlaylistId, TrackId FROM playlist_track;
 """, con=engine.connect())
aux=list(df_AuxTrackId['TrackId'])
auxArtist=[]
for i in auxAlbum:
    aux22=df_album.index[df_album['AlbumId'] == i].tolist()
    auxArtist.append(df_album['ArtistId'][aux22[0]])

df_Quantity=pd.read_sql_query("""SELECT Quantity FROM invoice_items;
 """, con=engine.connect())
df_Quantity.head()

# Invoice_Fact

In [ ]:
columnas=['InvoiceID','CustomerID','TimeID','LocationID','TrackID','PlaylistID','ArtistID','AlbumID','Quantity']
df = pd.DataFrame(list(zip(list(df_AuxInvoiceId['InvoiceId']),auxCustomer,auxTime,auxLocation,
                          list(df_AuxTrackId['TrackId']),auxPlaylist,auxArtist,auxAlbum,
                           list(df_Quantity['Quantity']))), columns = columnas)
updateData('Invoice_Fact',df,list(df.columns.values))

['InvoiceID', 'CustomerID', 'TimeID', 'LocationID', 'TrackID', 'PlaylistID', 'ArtistID', 'AlbumID', 'Quantity']
